<a href="https://colab.research.google.com/github/cfong32/netflix-prize/blob/main/step1_dataset_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import json
import csv
import pandas as pd
from google.colab import files
from tqdm import tqdm

In [2]:
# upload your kaggle.json (your API token)
print('upload your kaggle.json (your API token)')
files.upload()

# download netflix-prize-data from kaggle
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d netflix-inc/netflix-prize-data
!unzip netflix-prize-data.zip

upload your kaggle.json (your API token)


Saving kaggle.json to kaggle.json
 99% 675M/683M [00:04<00:00, 207MB/s]
100% 683M/683M [00:04<00:00, 164MB/s]
Archive:  netflix-prize-data.zip
  inflating: README                  
  inflating: combined_data_1.txt     
  inflating: combined_data_2.txt     
  inflating: combined_data_3.txt     
  inflating: combined_data_4.txt     
  inflating: movie_titles.csv        
  inflating: probe.txt               
  inflating: qualifying.txt          


In [3]:
# read the whole dataset 
# save it into a long dataframe format

inp_txts = [
    'combined_data_1.txt',
    'combined_data_2.txt',
    'combined_data_3.txt',
    'combined_data_4.txt'
]
outp_csv = 'ratings.csv'

movie_id2idx = {}    # a lookup from movie_id to movie_idx
user_id2idx  = {}    # a lookup from user_id to user_idx
new_movie_idx = 0    # new idx of movies
new_user_idx  = 0    # idx of users

with open(outp_csv, 'w') as file:
    writer = csv.writer(file)
    writer.writerow(['user', 'movie', 'rating'])

    for path in inp_txts:
        lines = open(path, 'r').read().split('\n')

        for line in tqdm(lines):
            if line.endswith(':'):
                movie_id = int(line[:-1])    # read movie_id
                i = new_movie_idx            # set i to new_movie_idx
                movie_id2idx[movie_id] = i   # update lookup
                assert len(movie_id2idx) == i+1
                new_movie_idx += 1

            elif line == '':
                continue

            else:
                user_id, rating, date = line.split(',')  # read a user's rating line
                user_id = int(user_id)
                rating  = int(rating)

                if user_id in user_id2idx:    # lookup user's idx
                    j = user_id2idx[user_id]
                else:
                    j = new_user_idx          # add one user
                    user_id2idx[user_id] = j
                    assert len(user_id2idx) == j+1
                    new_user_idx += 1

                writer.writerow([j, i, rating])

100%|██████████| 26851927/26851927 [01:28<00:00, 303791.08it/s]


In [5]:
# save original indexes
json.dump(movie_id2idx, open('movie_id2idx.json', 'w'))
json.dump(user_id2idx, open('user_id2idx.json', 'w'))

In [6]:
# check csv
df_ratings = pd.read_csv('ratings.csv')
df_ratings

,user,movie,rating
0,0,0,3
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,3
...,...,...,...
100480502,542,17769,4
100480503,29273,17769,3
100480504,29251,17769,1
100480505,22793,17769,4


In [9]:
# upload to my google drive
# from google.colab import drive
# drive.mount('/content/gdrive')
# !cp ratings.csv gdrive/MyDrive/Shared/
# !cp *id2idx.* gdrive/MyDrive/Shared/